## Phase: model iteration

After sharing my results with my colleagues at a meeting, the more senior data scientist suggested that I can try applying a log on the sales price because the price at the high end seem to grow rather exponentially.

I didn't want to edit my notebook in place, so I'm just going to copy them over and see the changes here.

In [ ]:
import lineapy

In [ ]:
cleaned_data = lineapy.get("cleaned_data_housing").value

In [1]:
import pickle

with open("../without_linea/cleaned_data_df", 'rb') as pickle_file:
    cleaned_data = pickle.load(pickle_file)

In [2]:
cleaned_data = cleaned_data.dropna()

In [3]:
import numpy as np

cleaned_data['SalesPriceLog'] = np.log(cleaned_data.loc[:, 'SalePrice'])

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model as lm

In [5]:
train, val = train_test_split(cleaned_data, test_size=0.3, random_state=42)
X_train = train.drop(['SalesPriceLog'], axis = 1)
y_train = train.loc[:, 'SalesPriceLog']
X_val = val.drop(['SalesPriceLog'], axis = 1)
y_val = val.loc[:, 'SalesPriceLog']

In [6]:
linear_model = lm.LinearRegression(fit_intercept=True)
linear_model.fit(X_train, y_train)
y_fitted = linear_model.predict(X_train)
y_predicted = linear_model.predict(X_val)

In [7]:
def rmse(predicted, actual):
    """
    Calculates RMSE from actual and predicted values
    Input:
      predicted (1D array): vector of predicted/fitted values
      actual (1D array): vector of actual values
    Output:
      a float, the root-mean square error
    """
    return np.sqrt(np.mean((np.exp(actual) - np.exp(predicted))**2))
rmse(y_predicted, y_val)

43597.568155385394

In [ ]:
(43597.568155385394 - 44444.37608032081) / 44444.37608032081

## Conclusion

We find that this does decrease the error rate by about 1 percent.

In [ ]:
model_artifact = lineapy.save(linear_model, "linear_model_housing_with_log_transform")

In [ ]:
model_artifact.to_pipeline()()